# Simulate travel time between two points for different transport modes

In [1]:
import haversine as hs
import folium
import math
import sys
from pathlib import Path
from station_utilities import *
from simulations import *
from tqdm.notebook import tqdm

In [2]:
def init_stationfinder() -> StationFinder:
    """
    Initializes the StationFinder utility by loading Citibike and subway graph data.

    Returns:
        StationFinder: An initialized StationFinder object.
    
    Raises:
        SystemExit: If required graph files (.gml) are not found.
    """
    citibike_weekday = '../citibike_weekday_network.gml'
    citibike_weekend = '../citibike_weekend_network.gml'
    subway_graph = '../subway_graph_weekday_weekend.gml'

    citibike_path = citibike_weekday if Path(citibike_weekday).exists() else citibike_weekend

    if not Path(citibike_path).exists():
            print("Error: Citibike graph not found. Please run citibike_processor.py first.")
            sys.exit(1)
        
    if not Path(subway_graph).exists():
        print(f"Error: Subway graph not found at {subway_graph}")
        sys.exit(1)

    print("Initializing StationFinder...")
    try:
        finder = StationFinder(
            citibike_graph_path=citibike_path,
            subway_graph_path=subway_graph
        )
    except Exception as e:
        print(f"Error initializing StationFinder: {e}")
        print("\nIf subway coordinates are not available, you can:")
        print("1. Provide a CSV file with subway coordinates")
        print("2. Download MTA GTFS data manually from: http://web.mta.info/developers/data/nyct/subway/google_transit.zip")
        sys.exit(1)

    return finder

In [3]:
station_finder = init_stationfinder()

Initializing StationFinder...
Successfully loaded 379 subway stations
Successfully loaded 379 subway stations


In [4]:
def simulate(src_hub_center, dst_hub_center, station_finder, radius_km=1.0, num_points=100, departure_time=(8,0), virtual=False):
    m = None

    simulator = Simulator(finder=station_finder)

    src_hub_points = simulator.generate_random_points(src_hub_center, radius_km=radius_km, num_points=num_points)
    dst_hub_points = simulator.generate_random_points(dst_hub_center, radius_km=radius_km, num_points=num_points)

    if virtual:
        print("Inserting virtual station at source hub center...")
        simulator.insert_station(('virtual-station', src_hub_center))

    # --- Calculate for Bike Transit ---
    print("" + "="*80)
    print('Simulating Citibike journeys...')
    print("" + "="*80)
    bike_travel_time_matrix, bike_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, radius_km=radius_km, mode='citibike', departure_time=departure_time)

    # --- Calculate for Subway Transit ---
    subway_travel_time_matrix, subway_route_matrix = simulator.calculate_travel_time_matrix(src_hub_points, dst_hub_points, radius_km=radius_km, mode='subway', departure_time=departure_time)

    results = {
        'bike_travel_time_matrix': bike_travel_time_matrix,
        'bike_route_matrix': bike_route_matrix,
        'subway_travel_time_matrix': subway_travel_time_matrix,
        'subway_route_matrix': subway_route_matrix
    }

    # create a map visualization of the results
    # m = simulator.map_simulation_results(bike_route_matrix, subway_route_matrix, src_hub_points, dst_hub_points)

    return results, m

In [5]:
def calculate_average_travel_times(results):
    bike_tt_matrix, subway_tt_matrix = results['bike_travel_time_matrix'], results['subway_travel_time_matrix']

    # Calculate the mean of all values in the bike travel time matrix
    avg_bike_time_seconds = np.mean(bike_tt_matrix)

    # Calculate the mean of all values in the subway travel time matrix
    avg_subway_time_seconds = np.mean(subway_tt_matrix)

    return avg_bike_time_seconds, avg_subway_time_seconds

In [6]:
def run_simulation(src_hubs, dst_hub, radius_km=1.0, num_points=100, departure_time=(8,0), virtual=False):
    dst_hub_center = dst_hub[1]

    bike_results = dict()
    subway_results = dict()

    for i, hub in enumerate(src_hubs):
        hub_coords = src_hubs[hub]
        print("" + "="*80)
        print(f"\nRunning simulations for source hub {i+1}/{len(src_hubs)} ({hub}) at location: {hub_coords}\n")
        print("" + "="*80)
        results, m = simulate(hub_coords, dst_hub_center, station_finder, radius_km=radius_km, num_points=num_points, departure_time=departure_time, virtual=virtual)
        avg_bike_time, avg_subway_time = calculate_average_travel_times(results)
        
        avg_bike_time_seconds = f"{int(round(avg_bike_time % 60)):02d}"
        avg_subway_time_seconds = f"{int(round(avg_subway_time % 60)):02d}"

        bike_results[hub] = f"{math.floor(avg_bike_time / 60)}:{avg_bike_time_seconds}"
        subway_results[hub] = f"{math.floor(avg_subway_time / 60)}:{avg_subway_time_seconds}"

    bike_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Bike': bike_results})
    subway_avg_times = pd.DataFrame.from_dict({'Average Travel Time By Metro': subway_results})
    print("" + "="*80)
    print(f"\nSimulation for dst {dst_hub[0]} complete!\n")
    print("" + "="*80)

    return bike_avg_times, subway_avg_times

In [7]:
src_hubs = {
    "Roosevelt Island Coast": (40.7699167, -73.9278056),
    "Steinway St / 31 Av": (40.7612778, -73.9171111),
    "North Brooklyn (Gates / Throop Av)": (40.6872500, -73.9417778),
    "Brooklyn Navy Yard (Flushing / Navy St)": (40.6982778, -73.9805556),
    "South Bronx (161 St / Melrose Av)": (40.8241389, -73.9139167),
    "South Manhattan (Houston / Clinton St)": (40.7214444, -73.9838611),
    "South Manhattan (Ave A / St. Marks Pl)": (40.7260556, -73.9835000)
}

dst_hubs = {
    "Little Italy / Chinatown": (40.7194167, -73.9975556),
    "Central Park Entrance / Columbus Monument": (40.7678889, -73.9820556),
    "The Bronx (Residential)": (40.8568889, -73.8877222),
    "Astoria Park (North Roosevelt Island Coast)": (40.7818333, -73.9173056),
    "Red Hook (Brooklyn Bay Area)": (40.6739722, -74.0081111),
    "North Queens / Jackson Heights": (40.7436667, -73.8871111),
    "West Queens / North Brooklyn": (40.7123889, -73.9440278)
}

## Simulate Baseline (No virtual station)

In [ ]:
radius_km = 1.0
num_points = 10
departure_time = (12, 0) # 12PM
virtual = False

os.makedirs('simulation_results/baseline', exist_ok=True)

for dst_hub in tqdm(dst_hubs, total=len(dst_hubs), desc='Simulating travels to destination hubs:'):
    bikes_results_df, subway_results_df = run_simulation(src_hubs, (dst_hub, dst_hubs[dst_hub]), radius_km=radius_km, num_points=num_points, departure_time=departure_time, virtual=virtual)
    print(bikes_results_df.head())
    print(subway_results_df.head())
    dst_hub_name = f'{dst_hub.replace(" / ", "-").replace(" ", "_")}'
    save_file_path = f'simulation_results/baseline/{dst_hub_name}' if not virtual else f'simulation_results/with_virtual/{dst_hub_name}'
    merged_df = bikes_results_df.join(subway_results_df)
    merged_df.to_csv(f'{save_file_path}.csv')

# # print("--- Average Travel Times for All Pairs ---")
# # print(f"Average Bike Travel Time: {math.floor(avg_bike_time_seconds / 60)} minutes and {avg_bike_time_seconds % 60:.0f} seconds")
# # print(f"Average Subway Travel Time: {math.floor(avg_subway_time_seconds / 60)} minutes and {avg_subway_time_seconds % 60:.0f} seconds")

Simulating travels to destination hub::   0%|          | 0/7 [00:00<?, ?it/s]


Running simulations for source hub 1/7 (Roosevelt Island Coast) at location: (40.7699167, -73.9278056)

Simulating Citibike journeys...


Calculating subway travel times:   0%|          | 0/100 [00:00<?, ?it/s]


Running simulations for source hub 2/7 (Steinway St / 31 Av) at location: (40.7612778, -73.9171111)

Simulating Citibike journeys...


Calculating subway travel times:   0%|          | 0/100 [00:00<?, ?it/s]

...


Running simulations for source hub 6/7 (South Manhattan (Houston / Clinton St)) at location: (40.7214444, -73.9838611)

Simulating Citibike journeys...


Calculating subway travel times:   0%|          | 0/100 [00:00<?, ?it/s]


Running simulations for source hub 7/7 (South Manhattan (Ave A / St. Marks Pl)) at location: (40.7260556, -73.9835)

Simulating Citibike journeys...


Calculating subway travel times:   0%|          | 0/100 [00:00<?, ?it/s]


Simulation for dst West Queens / North Brooklyn complete!

                                        Average Travel Time By Bike
Roosevelt Island Coast                                        33:21
Steinway St / 31 Av                                           32:28
North Brooklyn (Gates / Throop Av)                            17:35
Brooklyn Navy Yard (Flushing / Navy St)                       33:51
South Bronx (161 St / Melrose Av)                             66:26
                                        Average Travel Time By Metro
Roosevelt Island Coast                                         48:34
Steinway St / 31 Av                                            45:12
North Brooklyn (Gates / Throop Av)                             35:31
Brooklyn Navy Yard (Flushing / Navy St)                        46:30
South Bronx (161 St / Melrose Av)                              56:26


## Simulate with virtual stations

In [ ]:
radius_km = 0.5
num_points = 10
departure_time = (12, 0) # 12AM
virtual = True

os.makedirs('simulation_results/with_virtual', exist_ok=True)

for dst_hub in tqdm(dst_hubs, total=len(dst_hubs), desc='Simulating travels to destination hub:'):
    bikes_results_df, subway_results_df = run_simulation(src_hubs, (dst_hub, dst_hubs[dst_hub]), radius_km=radius_km, num_points=num_points, departure_time=departure_time, virtual=virtual)
    print(bikes_results_df)
    print(subway_results_df)
    dst_hub_name = f'{dst_hub.replace(" / ", "-").replace(" ", "_")}'
    save_file_path = f'simulation_results/baseline/{dst_hub_name}' if not virtual else f'simulation_results/with_virtual/{dst_hub_name}'
    merged_df = bikes_results_df.join(subway_results_df)
    merged_df.to_csv(f'{save_file_path}.csv')

# print("--- Average Travel Times for All Pairs ---")
# print(f"Average Bike Travel Time: {math.floor(avg_bike_time_seconds / 60)} minutes and {avg_bike_time_seconds % 60:.0f} seconds")
# print(f"Average Subway Travel Time: {math.floor(avg_subway_time_seconds / 60)} minutes and {avg_subway_time_seconds % 60:.0f} seconds")

Simulating travels to destination hub::   0%|          | 0/7 [00:00<?, ?it/s]


Running simulations for source hub 1/7 (Roosevelt Island Coast) at location: (40.7699167, -73.9278056)

Inserting virtual station at source hub center...
Inserting new station: virtual-station at (40.7699167, -73.9278056)
Simulating Citibike journeys...


Calculating subway travel times:   0%|          | 0/100 [00:00<?, ?it/s]